In [3]:
!pip install iterative-stratification

In [11]:
# Imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import cv2
import os
import sys
from io import StringIO
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import tensorflow.keras.applications.resnet50 as resnet
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import Sequence
import numpy as np, pandas as pd, os, cv2, gc
import warnings
warnings.filterwarnings('ignore')

In [27]:
print(os.listdir("/kaggle/input"))

# Path to competition dataset
data_dir = "/kaggle/input/grand-xray-slam-division-b"
# Check what files are inside
print('Filenames of the data', os.listdir(data_dir))

['grand-xray-slam-division-b']
Filenames of the data ['test2', 'sample_submission_2.csv', 'train2.csv', 'train2']


In [28]:
# Load the training CSV metadata with labels
train = pd.read_csv("/kaggle/input/grand-xray-slam-division-b/train2.csv")

print('Metadata shape:',train.shape)
train.head()

Metadata shape: (108494, 21)


,Image_name,Patient_ID,Study,Sex,Age,ViewCategory,ViewPosition,Atelectasis,Cardiomegaly,Consolidation,...,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,00000003_001_001.jpg,3,1,Male,41.0,Frontal,AP,0,1,0,...,1,0,0,1,0,0,0,0,0,0
1,00000004_001_001.jpg,4,1,Female,20.0,Frontal,PA,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,00000004_001_002.jpg,4,1,Female,20.0,Lateral,Lateral,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,00000006_001_001.jpg,6,1,Female,42.0,Frontal,AP,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,00000010_001_001.jpg,10,1,Female,50.0,Frontal,PA,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [21]:
# # 🧹 Clean up columns before K-Fold split
# drop_cols = ["Patient_ID", "Study"]     # irrelevant for model
# train = train.drop(columns=drop_cols)

# print("✅ Dropped Patient_ID and Study columns.")
# print(f"Remaining columns: {list(train.columns)}")

✅ Dropped Patient_ID and Study columns.
Remaining columns: ['Image_name', 'Sex', 'Age', 'ViewCategory', 'ViewPosition', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']


In [29]:
# 1. Feature & Target Preperation
# Define labels
conditions = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
    'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
    'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'
]
# Features you want
features = ["ViewCategory", "ViewPosition", "Age", "Sex"]

# Encode categorical features
from sklearn.preprocessing import LabelEncoder

train_enc = train.copy()   # train data encoded
for col in ["ViewCategory", "ViewPosition", "Sex"]:  # features that can be encoded
    le = LabelEncoder()
    train_enc[col] = le.fit_transform(train[col].astype(str))

X = train_enc[features].values
y = train[conditions].values

In [30]:
print(X.shape) # 4 features (ViewCategory, ViewPosition, Age, Sex)
print(y.shape)  # 14 conditions

(108494, 4)
(108494, 14)


In [31]:
# 2. Adding ViewBalancing for Stratification: ViewCategory= Frontal, Lateral; since ViewCategory is unbalanced

# One-hot encode ViewCategory and append to 
view_onehot = pd.get_dummies(train["ViewCategory"], prefix="view").values

y_aug = np.hstack([y, view_onehot])  # augmented target matrix (added ViewCategory as y to stratify and reduce bias)

In [32]:
# #3 
# # =============================================================
# # ⚖️ Multi-Label Stratified K-Fold Split (balances rare diseases + view angles)
# # =============================================================

# mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# folds = []  # store train/val DataFrames

# for fold, (train_idx, val_idx) in enumerate(mskf.split(X, y_aug)):
#     print(f"\n================ FOLD {fold+1} ================")
#     train_df = train.iloc[train_idx].reset_index(drop=True)
#     val_df   = train.iloc[val_idx].reset_index(drop=True)
#     folds.append((train_df, val_df))

#     # Check label + view balance
#     print("  Train views:", train_df["ViewCategory"].value_counts().to_dict())
#     print("  Val views:", val_df["ViewCategory"].value_counts().to_dict())
#     print("  Train label sums:", train_df[conditions].sum().to_dict())
#     print("  Val label sums:", val_df[conditions].sum().to_dict())
#     print("-"*60)

# # ✅ Use first fold for training (you can change to fold[1], fold[2], etc.)
# train_df, val_df = folds[0]
# print(f"\n✅ Selected Fold 1 — Train {train_df.shape}, Val {val_df.shape}")
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# =============================================================
# 🧹 Clean dataset: drop identifiers
# =============================================================
drop_cols = ["Study"]
train = train.drop(columns=drop_cols)
print("✅ Dropped 'Study' column. Keeping 'Patient_ID' temporarily for grouping.")

# =============================================================
# 🧩 Hybrid Multi-Label + Patient-Level Split
# =============================================================
unique_patients = train["Patient_ID"].unique()
patient_groups = train.groupby("Patient_ID")[conditions].max().reset_index()

# Stratify based on each patient’s condition profile
y_patient = patient_groups[conditions].values
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

folds = []
for fold, (train_idx, val_idx) in enumerate(mskf.split(unique_patients, y_patient)):
    train_patients = unique_patients[train_idx]
    val_patients   = unique_patients[val_idx]

    train_df = train[train["Patient_ID"].isin(train_patients)].reset_index(drop=True)
    val_df   = train[train["Patient_ID"].isin(val_patients)].reset_index(drop=True)

    folds.append((train_df, val_df))

    print(f"\n================ FOLD {fold+1} ================")
    print(f" Train patients: {len(train_patients)} | Val patients: {len(val_patients)}")
    print(f" Train images: {train_df.shape[0]} | Val images: {val_df.shape[0]}")
    print("  Train views:", train_df["ViewCategory"].value_counts().to_dict())
    print("  Val views:", val_df["ViewCategory"].value_counts().to_dict())
    print("-"*60)

# ✅ Select first fold for training
train_df, val_df = folds[0]

# Drop Patient_ID now that split is done
train_df = train_df.drop(columns=["Patient_ID"])
val_df = val_df.drop(columns=["Patient_ID"])

print(f"\n✅ Selected Fold 1 — Train {train_df.shape}, Val {val_df.shape}")


✅ Dropped 'Study' column. Keeping 'Patient_ID' temporarily for grouping.

================ FOLD 1 ================
 Train patients: 25704 | Val patients: 6373
 Train images: 87032 | Val images: 21462
  Train views: {'Frontal': 76209, 'Lateral': 10823}
  Val views: {'Frontal': 18805, 'Lateral': 2657}
------------------------------------------------------------

================ FOLD 2 ================
 Train patients: 25609 | Val patients: 6468
 Train images: 86566 | Val images: 21928
  Train views: {'Frontal': 75814, 'Lateral': 10752}
  Val views: {'Frontal': 19200, 'Lateral': 2728}
------------------------------------------------------------

================ FOLD 3 ================
 Train patients: 25672 | Val patients: 6405
 Train images: 86668 | Val images: 21826
  Train views: {'Frontal': 75938, 'Lateral': 10730}
  Val views: {'Frontal': 19076, 'Lateral': 2750}
------------------------------------------------------------

================ FOLD 4 ================
 Train patients: 2

In [33]:
train_df.head()

,Image_name,Sex,Age,ViewCategory,ViewPosition,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,00000003_001_001.jpg,Male,41.0,Frontal,AP,0,1,0,1,1,0,0,1,0,0,0,0,0,0
1,00000004_001_001.jpg,Female,20.0,Frontal,PA,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,00000004_001_002.jpg,Female,20.0,Lateral,Lateral,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,00000006_001_001.jpg,Female,42.0,Frontal,AP,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,00000010_001_001.jpg,Female,50.0,Frontal,PA,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [34]:

# =============================================================
# ⚖️ Weighted Data Generator for Rare Classes
# =============================================================
class XRayDataGenerator(Sequence):
    def __init__(self, dataframe, batch_size=32, img_size=(224, 224), is_test=False):
        self.df = dataframe.reset_index(drop=True)
        self.batch_size = batch_size
        self.img_size = img_size
        self.is_test = is_test
        self.conditions = conditions
        self.image_dir = (
            "/kaggle/input/grand-xray-slam-division-b/train2/"
            if not is_test
            else "/kaggle/input/grand-xray-slam-division-b/test2/"
        )

        # Compute inverse-frequency sample weights   NEW
        label_freq = self.df[self.conditions].sum().values + 1e-6
        inv_freq = 1. / label_freq
        self.sample_weights = (self.df[self.conditions].values * inv_freq).sum(axis=1)
        self.sample_weights /= self.sample_weights.sum()

    def __len__(self):
        return len(self.df) // self.batch_size

    def __getitem__(self, idx):
        idxs = np.random.choice(len(self.df), self.batch_size, p=self.sample_weights)
        batch = self.df.iloc[idxs]
        images, labels = [], []

        for _, row in batch.iterrows():
            img_path = os.path.join(self.image_dir, row["Image_name"])
            img = cv2.imread(img_path)
            img = cv2.resize(img, self.img_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img / 255.0
            images.append(img)
            if not self.is_test:
                labels.append(row[self.conditions].values.astype(np.float32))

        X = np.array(images, dtype=np.float32)
        if self.is_test:
            return X
        else:
            y = np.array(labels, dtype=np.float32)
            return X, y

# =============================================================
# 🧮 Asymmetric Loss (for Long-Tailed Distribution)   NEW
# =============================================================
def asymmetric_loss(gamma_pos=0, gamma_neg=4, clip=0.05):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, clip, 1 - clip)
        pos_loss = y_true * tf.math.log(y_pred) * (1 - y_pred) ** gamma_pos
        neg_loss = (1 - y_true) * tf.math.log(1 - y_pred) * y_pred ** gamma_neg
        return -tf.reduce_mean(pos_loss + neg_loss)
    return loss

# =============================================================
# 🧱 ResNet50 + Custom Head
# =============================================================
def build_resnet_model(num_classes=14):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation="sigmoid")(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

model = build_resnet_model()
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=asymmetric_loss(),
    metrics=[tf.keras.metrics.AUC(name="AUC")]
)
print("✅ Model compiled successfully.")

✅ Model compiled successfully.


In [ ]:
# =============================================================
# ⚙️ Callbacks (robust against val_auc / val_AUC mismatch)
# =============================================================
callbacks = [
    ModelCheckpoint("best_resnet_model.h5", monitor="val_auc", mode="max",
                    save_best_only=True, verbose=1),
    EarlyStopping(monitor="val_auc", patience=3, mode="max",
                  restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_auc", factor=0.3, patience=2,
                      mode="max", min_lr=1e-6, verbose=1)
]

# Backup monitors to catch both metric names:
alt_callbacks = [
    ModelCheckpoint("best_resnet_model.h5", monitor="val_AUC", mode="max",
                    save_best_only=True, verbose=1),
    EarlyStopping(monitor="val_AUC", patience=3, mode="max",
                  restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_AUC", factor=0.3, patience=2,
                      mode="max", min_lr=1e-6, verbose=1)
]

# =============================================================
# 🚀 Training
# =============================================================
EPOCHS = 8
BATCH_SIZE = 32
train_gen = XRayDataGenerator(train_df, batch_size=BATCH_SIZE)
val_gen   = XRayDataGenerator(val_df, batch_size=BATCH_SIZE)

print("🚀 Starting training...")
try:
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1
    )
except Exception as e:
    print("⚠️ Switching to alternate callback metric (val_AUC).")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        callbacks=alt_callbacks,
        verbose=1
    )

# =============================================================
# 🧾 Evaluate & Save Model
# =============================================================
hist = history.history
val_auc = hist.get("val_auc", [0])[-1] or hist.get("val_AUC", [0])[-1]
print(f"✅ Final Validation AUC: {val_auc:.4f}")

model.save("resnet50_final_longtail.h5")
print("✅ Model saved as resnet50_final_longtail.h5")

gc.collect()


🚀 Starting training...
Epoch 1/8
  14/2719 ━━━━━━━━━━━━━━━━━━━━ 2:32:37 3s/step - AUC: 0.4409 - loss: 0.5340